In [ ]:
#! ollama pull  mistral:instruct

In [24]:
local_llm="mistral:instruct"

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_mistralai  import MistralAIEmbeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings


In [26]:
from dotenv import load_dotenv
load_dotenv()

True

Création index

In [49]:
#Récupération données (page web) servant de correction au RAG
url="https://france3-regions.francetvinfo.fr/auvergne-rhone-alpes/puy-de-dome/clermont-ferrand/elections-legislatives-2024-majorite-absolue-et-relative-cohabitation-motion-de-censure-le-vocabualire-pour-bien-comprendre-le-fonctionnement-de-l-assemblee-nationale-2993165.html"
loader=WebBaseLoader(url)
docs=loader.load()

#découpage données
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=100
)
all_splits = text_splitter.split_documents(docs)

#representation vectorielle
embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

#base de données vectorielles
vectorstore=Chroma.from_documents(
    documents=all_splits,
    collection_name="rag_chroma",
    embedding=embedding
)

retriever=vectorstore.as_retriever()


In [50]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser

# LLM
llm = ChatOllama(model=local_llm, format="json", temperature=0)

prompt = PromptTemplate(
    template="""Vous êtes un évaluateur évaluant la pertinence d'un document récupéré par rapport à une question d'un utilisateur. \n 
    Voici le document récupéré : \n\n {document} \n\n
    Voici la question de l'utilisateur : {question} \n
    Si le document contient des mots-clés liés à la question de l'utilisateur, notez-le comme pertinent. \n
    Il n est pas nécessaire que ce soit un test rigoureux. Le but est de filtrer les récupérations erronées. \n
    Donnez un score binaire 'yes' ou 'no' pour indiquer si le document est pertinent par rapport à la question. \n
    Fournissez le score binaire au format JSON avec un « score » à clé unique et sans prémisse ni explication.""",
    input_variables=["question", "document"],
)

retrieval_grader = prompt | llm | JsonOutputParser()
question = "Comment fonctionne une cohabitation ?"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
score = retrieval_grader.invoke({"question": question, "document": doc_txt})

In [51]:
score

{'score': 'yes'}

Graph State


In [52]:
from typing import Annotated,TypedDict,Dict

from langchain_core.messages import BaseMessage
from langchain.schema import Document

class GraphState(TypedDict):
    """
    Représente l'état de Nontre graphique
    
    Attributes: 
        keys: Un dictionnaire où chaque clé est une chaîne.
    """

    keys: Dict[str,any]

In [53]:
from langchain_community.chat_models import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mistralai.chat_models import ChatMistralAI
from langchain import hub
from langchain.schema import Document

In [54]:
def retrieve(state):
    """
    Récupère les documents

    Args:
        state (dict): L'état actuel du graphique

    Returns:
        state (dict): Nouvelle clé ajoutée à l'état, documents, qui contient les documents récupérés
    """
    print("---RETRIEVE---")
    state_dict=state["keys"]
    question = state_dict["question"]

    # Récupération
    documents = retriever.get_relevant_documents(question)
    return {"keys":{"documents": documents, "question": question}}


def generate(state):
    """
    Génère la réponse 

    Args:
        state (dict): L'état actuel du graphique

    Returns:
        state (dict): Nouvelle clé ajoutée à l'état, génération, qui contient la génération
    """
    print("---GENERATE---")
    state_dict= state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]

    prompt=hub.pull("rlm/rag-prompt")

    llm= ChatOllama(model=local_llm, temperature=0)


    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    rag_chain = prompt | llm | StrOutputParser()

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {
        "keys": {"documents": documents, 
                 "question": question, 
                 "generation": generation}
    }


def grade_documents(state):
    """
    Détermine si les documents récupérés sont pertinents pour la question.

    Args:
        state (dict): L'état actuel du graphique

    Returns:
        state (dict): Met à jour la clé des documents avec uniquement les documents pertinents filtrés
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    state_dict= state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]

    llm= ChatOllama(model=local_llm,format="json",temperature=0)

    prompt = PromptTemplate(
        template="""Vous êtes un évaluateur évaluant la pertinence d'un document récupéré par rapport à une question d'un utilisateur. \n
            Voici le document récupéré: \n\n {document} \n\n
            Voici la question de l'utilisateur: {question} \n
            Si le document contient des mots-clés liés à la question de l'utilisateur, notez-le comme pertinent. \n
            Il n est pas nécessaire que ce soit un test rigoureux. Le but est de filtrer les récupérations erronées. \n
            Donnez un score binaire 'Yes' ou 'No' pour indiquer si le document est pertinent par rapport à la question. \n
            Fournissez le score binaire au format JSON avec un « score » à clé unique et une explication ou une explication non premable.""",
        input_variables=["question", "document"],
    )

    retrieval_grader = prompt | llm | JsonOutputParser()

    # Score chaque document
    filtered_docs = []
    search ="No"
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score["score"]
        if grade == "Yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            search ="Yes"
            continue
    return {
        "keys": {
            "documents": filtered_docs, 
            "question": question,
            "run_web_search":search,
            
            }
    }


def transform_query(state):
    """
    Transforme la requête pour produire une meilleure question.

    Args:
        state (dict): L'état actuel du graphique

    Returns:
        state (dict): Met à jour la clé de question avec une question reformulée
    """

    print("---TRANSFORM QUERY---")
    state_dict= state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]

    # Ré-écriture de la question
    re_write_prompt = PromptTemplate(
        template="""Vous êtes un rédacteur de questions qui convertit une question d'entrée en une meilleure version optimisée \n
            pour la récupération du vectorstore. Examinez l'entrée et essayez de raisonner sur l'intention sémantique sous-jacente et de formuler une question améliorée. \n
            Voici la question initiale: \n\n {question}. Question améliorée sans préambule: \n """,
    input_variables=["question"]
    )

    llm=ChatOllama(model=local_llm, temperature=0)

    better_question_chain = re_write_prompt | llm | StrOutputParser()
    better_question = better_question_chain.invoke({"question": question})
    
    return {
        "keys":{
            "documents": documents, 
            "question": better_question}}


### Edges
def web_search(state):
    """
    Recherche sur le Web basée sur la question reformulée en utilisantTavily API.

    Args:
        state (dict): L'état actuel du graphique

    Returns:
        state (dict): Résultats Web annexés aux documents
    """

    print("---WEB SEARCH---")
    state_dict= state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]

    # Web search
    web_search_tool=TavilySearchResults(k=5)
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    documents.append(web_results)

    return {
        "keys":{
            "documents": documents, 
            "question": question}
    }


def decide_to_generate(state):
    """
    Détermine s'il faut générer une réponse ou régénérer une question pour une recherche sur le Web.

    Args:
        state (dict): L'état actuel du graphique de l'agent, y compris toutes les clés.

    Returns:
        str: Next Node to call
    """

    print("---DECIDE TO GENERATE---")
    state_dict= state["keys"]
    question = state_dict["question"]
    filtered_documents = state_dict["documents"]
    search= state_dict["run_web_search"]

    if search =="Yes":
        
        print(
            "---DECISION: TRANSFORM QUERY and RUN WEB SEARCH---"
        )
        return "transform_query"
    else:
        
        print("---DECISION: GENERATE---")
        return "generate"



In [55]:
import pprint
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Definition des noeuds
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("transform_query", transform_query)  # transform_query
workflow.add_node("web_search", web_search)  # web search

# Construction du graphique
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "web_search")
workflow.add_edge("web_search", "generate")
workflow.add_edge("generate", END)

# Compilation
app = workflow.compile()

In [56]:
# Test
inputs = {
    "keys":{
        "question": "Comment fonctionne une cohabitation ?"}
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Noeud
        pprint(f"Noeud '{key}':")
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")



---RETRIEVE---
"Noeud 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---DECIDE TO GENERATE---
---DECISION: GENERATE---
"Noeud 'grade_documents':"
'\n---\n'
---GENERATE---
"Noeud 'generate':"
'\n---\n'


In [57]:
print("\n Réponse :",value["keys"]["generation"])


 Réponse :  The document contains information about the French legislative elections of 2024 and the terms "cohabitation", "majority absolute", "majority relative", and "motion of censure".

Cohabitation refers to a situation where a president and a parliament have different political colors, meaning they belong to different parties or ideologies. This was the case in 1986 when François Mitterrand, a president of the left, had to cohabit with a right-wing majority led by Jacques Chirac. In 1993, the same scenario occurred with Balladur taking office at Matignon after Chirac had already given his position. The term was also used in 1997 when the opposite situation happened, with a president of the right, Chirac, dissolving the parliament and finding himself with a left-wing majority. Emmanuel Macron may find himself in a similar situation in 2024.

A motion of censure is a vote of no confidence in the government. It can be initiated by any member of the National Assembly and requires a

In [59]:
#Test 2
inputs = {
    "keys":{
        "question": "quel est le nom du personnage principal dans le film sorti écemment Dune ?"}
}
for output in app.stream(inputs):
    for key, value in output.items():
         pprint(f"Node '{key}':")
        
    pprint("\n---\n")



print("\n Réponse :",value["keys"]["generation"])

---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---DECIDE TO GENERATE---
---DECISION: TRANSFORM QUERY and RUN WEB SEARCH---
"Node 'grade_documents':"
'\n---\n'
---TRANSFORM QUERY---
"Node 'transform_query':"
'\n---\n'
---WEB SEARCH---
"Node 'web_search':"
'\n---\n'
---GENERATE---
"Node 'generate':"
'\n---\n'

 Réponse :  Le personnage principal dans le récent film "Dune" est Timothée Chalamet, qui joue le rôle de Leto Atréides.
